In [1]:
import polars as pl
from transformers import BartTokenizerFast

In [4]:
df = pl.read_parquet('/kaggle/input/nepbart-chunked-tokenized-masked/nepbart_tokenized_masked_train.parquet')

In [5]:
len(df)

2248625

In [12]:
tokenizer = BartTokenizerFast.from_pretrained("/kaggle/input/nepbart-tokenizer/nepbart_tokenizer")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


In [13]:
SEQUENCE_LENGTH = 512
CHUNK_LENGTH = 510

In [14]:
def add_padding(x, tokenizer, seq_len):
    # print(type(x))
    sequence = [tokenizer.bos_token_id] + x.to_list()  + [tokenizer.eos_token_id]

    assert len(sequence) <= seq_len

    if len(sequence) == seq_len:
        return sequence
    
    final_sequence = sequence + [tokenizer.pad_token_id] * (seq_len - len(sequence))
    # print(final_sequence)
    # print(type(final_sequence))
    # return pl.Series(None, [final_sequence], dtype=pl.List(pl.UInt16))
    return final_sequence

In [15]:
add_padding(df['labels'][0], tokenizer, 512)

[0,
 4476,
 24,
 22127,
 7680,
 24,
 6809,
 558,
 9037,
 4888,
 2139,
 1727,
 4537,
 5793,
 16242,
 4155,
 5793,
 1058,
 20275,
 18075,
 31400,
 2546,
 20770,
 2602,
 23587,
 7774,
 4376,
 2102,
 319,
 3161,
 10087,
 4537,
 558,
 28239,
 24,
 1235,
 15290,
 4157,
 12072,
 1643,
 2305,
 26597,
 24,
 5691,
 339,
 2331,
 558,
 16907,
 10605,
 2196,
 3892,
 1332,
 8085,
 592,
 16329,
 13758,
 9891,
 3518,
 667,
 16880,
 2115,
 815,
 705,
 906,
 5109,
 10643,
 10394,
 776,
 566,
 664,
 28984,
 9891,
 3518,
 5159,
 547,
 6665,
 2115,
 815,
 5883,
 2624,
 592,
 13031,
 14029,
 699,
 1631,
 13311,
 569,
 2333,
 1224,
 556,
 5291,
 6539,
 22063,
 2233,
 783,
 29917,
 931,
 5159,
 5183,
 5980,
 1103,
 5127,
 15180,
 638,
 821,
 3394,
 5210,
 25001,
 13565,
 903,
 632,
 667,
 6241,
 364,
 1729,
 601,
 30153,
 642,
 24,
 9528,
 1048,
 28295,
 24,
 18318,
 658,
 5623,
 1578,
 3518,
 660,
 14934,
 7427,
 32719,
 3518,
 4531,
 12277,
 10046,
 549,
 684,
 632,
 1729,
 8661,
 8040,
 660,
 19353,
 13565

In [16]:
df = df.with_columns(pl.col('input_ids').map_elements(lambda x: [1] * (len(x)+2) + [0] * (SEQUENCE_LENGTH - len(x) - 2), return_dtype=pl.List(pl.UInt8)).alias('attention_mask'))

In [17]:
max(df.with_columns(pl.col('attention_mask').map_elements(lambda x: len(x), return_dtype=int).alias('len_'))['len_'])

512

In [18]:
df = df.with_columns(pl.col('labels').map_elements(lambda x: add_padding(x, tokenizer, SEQUENCE_LENGTH), return_dtype=pl.List(pl.UInt16)).alias('labels'))

In [19]:
df = df.with_columns(pl.col('input_ids').map_elements(lambda x: add_padding(x, tokenizer, SEQUENCE_LENGTH), return_dtype=pl.List(pl.UInt16)).alias('input_ids'))

In [20]:
df

labels,input_ids,attention_mask
list[u16],list[u16],list[u8]
"[0, 4476, … 2]","[0, 4476, … 2]","[1, 1, … 1]"
"[0, 12949, … 2]","[0, 4, … 2]","[1, 1, … 1]"
"[0, 10605, … 2]","[0, 10605, … 1]","[1, 1, … 0]"
"[0, 19839, … 1]","[0, 19839, … 1]","[1, 1, … 0]"
"[0, 8517, … 1]","[0, 8517, … 1]","[1, 1, … 0]"
…,…,…
"[0, 1122, … 1]","[0, 1122, … 1]","[1, 1, … 0]"
"[0, 4941, … 1]","[0, 4941, … 1]","[1, 1, … 0]"
"[0, 1314, … 1]","[0, 1314, … 1]","[1, 1, … 0]"


In [21]:
len(list(set(df.with_columns(pl.col('labels').map_elements(lambda x: len(x), return_dtype=int).alias('len_'))['len_'].to_list()))) == 1

True

In [22]:
len(list(set(df.with_columns(pl.col('input_ids').map_elements(lambda x: len(x), return_dtype=int).alias('len_'))['len_'].to_list()))) == 1

True

In [23]:
len(list(set(df.with_columns(pl.col('attention_mask').map_elements(lambda x: len(x), return_dtype=int).alias('len_'))['len_'].to_list()))) == 1

True

In [24]:
df

labels,input_ids,attention_mask
list[u16],list[u16],list[u8]
"[0, 4476, … 2]","[0, 4476, … 2]","[1, 1, … 1]"
"[0, 12949, … 2]","[0, 4, … 2]","[1, 1, … 1]"
"[0, 10605, … 2]","[0, 10605, … 1]","[1, 1, … 0]"
"[0, 19839, … 1]","[0, 19839, … 1]","[1, 1, … 0]"
"[0, 8517, … 1]","[0, 8517, … 1]","[1, 1, … 0]"
…,…,…
"[0, 1122, … 1]","[0, 1122, … 1]","[1, 1, … 0]"
"[0, 4941, … 1]","[0, 4941, … 1]","[1, 1, … 0]"
"[0, 1314, … 1]","[0, 1314, … 1]","[1, 1, … 0]"


In [25]:
df.write_parquet('nepbart_tokenized_masked_padded_test.parquet')

In [26]:
pl.read_parquet('/kaggle/working/nepbart_tokenized_masked_padded_test.parquet')

labels,input_ids,attention_mask
list[u16],list[u16],list[u8]
"[0, 4476, … 2]","[0, 4476, … 2]","[1, 1, … 1]"
"[0, 12949, … 2]","[0, 4, … 2]","[1, 1, … 1]"
"[0, 10605, … 2]","[0, 10605, … 1]","[1, 1, … 0]"
"[0, 19839, … 1]","[0, 19839, … 1]","[1, 1, … 0]"
"[0, 8517, … 1]","[0, 8517, … 1]","[1, 1, … 0]"
…,…,…
"[0, 1122, … 1]","[0, 1122, … 1]","[1, 1, … 0]"
"[0, 4941, … 1]","[0, 4941, … 1]","[1, 1, … 0]"
"[0, 1314, … 1]","[0, 1314, … 1]","[1, 1, … 0]"
